# Use two efficient approaches on computing Casimir energy

In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
import matplotlib.pyplot as plt
import krypy

## Standard Arnoldi method

In [4]:
N_wn = 20
lbound_wn = 0
ubound_wn = 10
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

h = 0.1
m = 20 # dimension of the Krylov subspace

In [5]:
for dist in np.linspace(0.5, 3, 11):
    results = []

    sphere1 = bempp.api.shapes.sphere(r = 0.5, h=h, origin=((dist/2) + 0.5, 0, 0))
    sphere2 = bempp.api.shapes.sphere(r = 1, h=h, origin=(-((dist/2) + 1), 0, 0))

    space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
    space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

    for index2, point2 in enumerate(quad_points):

            wavenumber = -1j * np.log(point2)

            slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
            slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
            slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
            slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

            mat11 = slp11.weak_form().A
            mat12 = slp12.weak_form().A
            mat21 = slp21.weak_form().A
            mat22 = slp22.weak_form().A
            mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
            mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

            mat = np.block([[mat11,mat12],[mat21,mat22]])  
            mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
            mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

            inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                                  [mat21_zero, mat22_inv]])  

            combined_inverse_free = inv_free_mat2.dot(mat)
            evals_inv_free, evect_inv_free = np.linalg.eig(combined_inverse_free)
            n = mat.shape[0]
            X_combined, H_combined = krypy.utils.arnoldi(combined_inverse_free, (np.ones(n)/np.linalg.norm(np.ones(n))).reshape((n,1)), maxiter = m,  ortho='dmgs')
            evals_H, evect_H = np.linalg.eig(H_combined[0:m, :])  

            method1_logdet = 0
            for i in range(len(evals_H)):
                method1_logdet += np.log(evals_H[i])

            results.append(method1_logdet)

    print(np.trapz(-np.array(results)/quad_points, quad_points)/(2*np.pi))

(0.02428258295892194+0j)
(0.012842834298928733+0j)
0.007160325201778342
(0.004460986722752892+0j)
(0.00301833223938047+0j)
(0.00214973140008303+0j)
(0.0015950855620079191+0j)
(0.0012209425782680166+0j)
0.0009581108995228483
0.0007677144605633957
(0.0006265539521859659+0j)
